# Hurto Personas

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
PATH_FILE = '/content/drive/MyDrive/Police-Data/Bronce'
OUTPUT_PATH_FILE = '/content/drive/MyDrive/Police-Data/Silver'
CODIGO_DANE_PATH = '/content/drive/MyDrive/Police-Data/dane_codes.xlsx'

## Libraries

In [3]:
import pandas as pd
import numpy as np
import os
import glob

## Read Files

In [4]:
def find_header_row(file_path):
    """
    Finds the header line with a column name
    """
    column_name = ['DEPARTAMENTO','DEPARTAMENTO ','Departamento','departamento']
    header_row = None

    df = pd.read_excel(file_path, header=None)

    for i, row in df.iterrows():
        if column_name[0] in row.values or column_name[1] in row.values or column_name[2] in row.values or column_name[3] in row.values:
            header_row = i
            break
        if i>20:
            break

    if header_row is None:
        raise ValueError(f"No se encontró la fila de encabezado en el archivo {file_path}")

    return header_row

def read_excel_with_variable_header(file_path):
    header_row = find_header_row(file_path)

    df = pd.read_excel(file_path, header=header_row)

    total_row_index = df[df.iloc[:, 0].str.contains("TOTAL", case=False, na=False)].index.min()

    if total_row_index is not None:
        df = df.iloc[:total_row_index]

    return df

In [5]:
folder = 'Hurto_Personas'
file_paths = glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xlsx')) + glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xls'))
dataframes = []

for file_path in file_paths:
    try:
        df = read_excel_with_variable_header(file_path)
        dataframes.append(df)
    except ValueError as e:
        print(e)

## Clean File

In [6]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA','FECHA  HECHO ':'FECHA','Fecha':'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO','Departamento':'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO','Municipio':'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE','Código DANE':'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'ARMA MEDIO ':'ARMAS_MEDIOS','Arma empleada':'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD','Cantidad':'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

standardized_dataframes = [standardize_columns(df) for df in dataframes]

In [7]:
hurto_personas_df = pd.concat(standardized_dataframes, ignore_index=True)

In [8]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','CONTUNDENTES','ESCOPOLAMINA','LLAVE MAESTRA','SIN EMPLEO DE ARMAS',
                             'CORTANTES','PERRO','PUNZANTES','JERINGA','ALUCINOGENOS','DIRECTA','PALANCAS','SUSTANCIAS TOXICAS','VEHICULO']
hurto_personas_df["AGRUPA_EDAD_PERSONA"] = hurto_personas_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
hurto_personas_df["GENERO"] = hurto_personas_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
hurto_personas_df["ARMAS_MEDIOS"] = hurto_personas_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [9]:
dane_codes = pd.read_excel(CODIGO_DANE_PATH,sheet_name='Municipios_dane')
def clean_data(data):
  temp = data.copy()
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].astype(str).str[:5]
  filtered_df = temp[temp['CODIGO_DANE'].apply(lambda x: x[-1].isdigit() if x else False)]
  filtered_df.loc[:, 'CODIGO_DANE'] = filtered_df['CODIGO_DANE'].astype(int)
  # Adjust upper case
  filtered_df = filtered_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)
  # fill NaN with most common value
  filtered_df['MUNICIPIO'] = filtered_df.groupby('CODIGO_DANE')['MUNICIPIO'].transform(lambda x: x.ffill().bfill())
  # Merge DANE CODE
  merged_df = filtered_df.merge(dane_codes, on='CODIGO_DANE', how='left', suffixes=('', '_csv'))
  # fill NaNs
  merged_df['MUNICIPIO'] = merged_df['MUNICIPIO'].fillna(merged_df['MUNICIPIO_csv'])
  merged_df = merged_df.drop(columns=['MUNICIPIO_csv'])
  merged_df.dropna(subset='MUNICIPIO',inplace=True)
  merged_df['CAPITAL'] = merged_df['MUNICIPIO'].apply(lambda municipio: 1 if municipio[-3:] == 'CT)' else 0)
  return merged_df

In [10]:
hurto_personas_filtered = clean_data(hurto_personas_df)

In [13]:
hurto_personas_output = hurto_personas_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

## Export

In [14]:
hurto_personas_output.to_csv(OUTPUT_PATH_FILE+"/hurto_personas.csv",index=False)